In [1]:
# Import all the libraries
import pandas as pd
import glob
import os
import re
from pprint import pprint

In [2]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\SC\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]


# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary', 'president'])
]


In [3]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\SC\20161108__sc__general__precinct.csv


In [4]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\SC\20160227__sc__primary__president__precinct.csv


In [5]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)
        
            
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()
        df["county"] = df["county"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
            df = df[~df['precinct'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]   

        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]               
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"].str.upper() == "PRESIDENT"]
            
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)



In [6]:
## Cleaning precinct, county name
# Fixing Cubbler
# pri_combined_df.loc[
#     ( 
#         pri_combined_df["county"].str.contains("\w*F120\w*", case=False, na=False)|
#         pri_combined_df["county"].str.contains("\w*STONE\w*", case=False, na=False)|
#         pri_combined_df["county"].str.contains("\w*OBBLER\w*", case=False, na=False),
#     "county")
# ] = "STONE"

In [7]:
# Checking any suspicious precinct, county
suspicious_precincts = pri_combined_df[
    pri_combined_df["precinct"].isna() |
    pri_combined_df["precinct"].str.strip().str.upper().str.contains(r"\w*TOTAL\w*", na=False) |
    pri_combined_df["precinct"].str.strip().str.upper().isin(["NONE", "NAN"]) |
    pri_combined_df["county"].isna() |
    pri_combined_df["county"].str.strip().str.upper().str.contains(r"\w*TOTAL\w*", na=False) |
    pri_combined_df["county"].str.strip().str.upper().isin(["NONE", "NAN"])
]

In [8]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]

primary_data

,precinct,party,candidate,votes
0,SALUDACENTENNIAL,DEM,Hillary Clinton,15
1,SALUDACLYDE,DEM,Hillary Clinton,0
2,SALUDADELMAR,DEM,Hillary Clinton,8
3,SALUDAFRUIT HILL,DEM,Hillary Clinton,130
4,SALUDAHIGGINS-ZOAR,DEM,Hillary Clinton,58
...,...,...,...,...
9566,BEAUFORTABSENTEE,DEM,Willie Wilson,7
9567,BEAUFORTEMERGENCY,DEM,Willie Wilson,0
9568,BEAUFORTFAILSAFE,DEM,Willie Wilson,0
9569,BEAUFORTPROVISIONAL,DEM,Willie Wilson,1


In [9]:
# Viewing candidate data
primary_data["candidate"].value_counts(dropna=False)

candidate
Hillary Clinton    2455
Bernie Sanders     2454
Willie Wilson      2337
Martin O'Malley    2325
Name: count, dtype: int64

In [10]:
# Cleaning Candidates

# Turning all primary data to uppercase
primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()

# unwanted_keywords = [
#     r"\w*VOTE\w*",
#     r"\w*UNCOM\w*",
#     r"\w*TOTAL\w*",
#     r"Blank Votes", r"All Others",r"Total Votes Cast",r"No Preference",
#     r"UNCOMMITTED"
# ]

# pattern = "|".join(unwanted_keywords)

# # # Assuming candidate column is already string and uppercase
# primary_data = primary_data[~primary_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

#fIXING DE LA FUENTE
# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("rocky.*fuente", case=False, na=False) |
#         primary_data["candidate"].str.contains("LA FUENTE", case=False, na=False) |
#         primary_data["candidate"].str.contains("\w*Fuque\w*", case=False, na=False)
#     ),
#     "candidate"
# ] = "LA FUENTE"


# primary_data.loc[:, "candidate"] = primary_data["candidate"].replace({
#     "Donald I. Trump": "Donald J. Trump",
# })

primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
# # Fixing the , candidate
# primary_data["candidate"] = (
#     primary_data["candidate"].str.split(",")
#     .str[0]
#     .str.strip()
#     )

# #Selecting only last name
primary_data["candidate"] = primary_data["candidate"].str.split().str[-1]

# Fixing Christie
primary_data.loc[
    primary_data["candidate"].str.contains("CHRISTLE", case=False, na=False),
    "candidate"
] = "CHRISTIE"

# Fixing KASICH
primary_data.loc[
    primary_data["candidate"].str.contains("KAISCH", case=False, na=False),
    "candidate"
] = "CHRISTIE"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("O'MALLEY", case=False, na=False),
    "candidate"
] = "O'MAILEY"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("PATAKL", case=False, na=False),
    "candidate"
] = "PATAKI"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("RUBLO", case=False, na=False),
    "candidate"
] = "RUBIO"

# # Fixing Huckabee
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ABEE\w*", case=False, na=False),
#     "candidate"
# ] = "HUCKABEE"

# # Fixing Fiorina
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ORINA\W*", case=False, na=False),
#     "candidate"
# ] = "FIORINA"

# # Fixing Gray
# primary_data.loc[
#     primary_data["candidate"].str.contains("GREY", case=False, na=False),
#     "candidate"
# ] = "GRAY"

# # Fixing Wilson
# primary_data.loc[
#     primary_data["candidate"].str.contains("WISON", case=False, na=False),
#     "candidate"
# ] = "WILSON"


# # Fixing separator 
# primary_data["candidate"] = (
#     primary_data["candidate"]
#     .str.split(r"\s*(?:and|/|&|–|-|\+)\s*", n=1, expand=True)[0]
#     .str.strip()
#     .str.upper()
# )

# # Fixing McMullin

# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("MCMULLIN", case=False, na=False) |
#         primary_data["candidate"].str.contains("EVAN MCMULLEN", case=False, na=False)
#     ),
#     "candidate"
# ] = "MCMULLIN"

# primary_data.loc[
#     primary_data["candidate"].str.contains("De La Fuen", case=False, na=False),
#     "candidate"
# ] = "FUENTE D"

## SPLITTINNG BY ,
# primary_data.loc[:,"candidate"] = (
#     primary_data["candidate"].str.split().str[0].str.upper()
# )

primary_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34304\2359132497.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34304\2359132497.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34304\2359132497.py:43: SettingWithCopyWarning: 
A value is trying to be set on a cop

candidate
CLINTON     2455
SANDERS     2454
WILSON      2337
O'MAILEY    2325
Name: count, dtype: int64

In [11]:
# Viewing Party
primary_data["party"].value_counts(dropna=False)

party
DEM    9571
Name: count, dtype: int64

In [12]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown

#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
# primary_data["party"] = primary_data.apply(
#     lambda row: fill_party_from_data(row, primary_data),
#     axis=1
# )

primary_data["party"].value_counts(dropna=False)


party
DEM    9571
Name: count, dtype: int64

In [13]:
# Cleaning Party
# Turning all general data party to uppercase
primary_data["party"] = primary_data["party"].astype(str).str.upper()

primary_data["party"] = primary_data.apply(
    lambda row: fill_party_from_data(row, primary_data),
    axis=1
)
primary_data["party"] = (
    primary_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEMOCRAT":"DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GREEN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK"
    })
)
primary_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34304\2220816968.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data["party"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34304\2220816968.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data.apply(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34304\2220816968.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

party
DEM    9571
Name: count, dtype: int64

In [14]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"]
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34304\2521670330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data.loc[:,"candidate_column"] = (


candidate_column,precinct,pri_dem_CLINTON,pri_dem_O'MAILEY,pri_dem_SANDERS,pri_dem_WILSON
0,ABBEVILLEABBEVILLE NO. 1,104,1,28,1
1,ABBEVILLEABBEVILLE NO. 2,193,0,24,1
2,ABBEVILLEABBEVILLE NO. 3,105,1,25,0
3,ABBEVILLEABBEVILLE NO. 4,53,0,5,0
4,ABBEVILLEABSENTEE,354,0,52,4
...,...,...,...,...,...
2450,YORKWATERSTONE,61,0,54,0
2451,YORKWINDJAMMER,79,1,87,0
2452,YORKWYLIE,37,0,29,0
2453,YORKYORK NO. 1,139,0,41,1


In [15]:
# Process general files
general_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)
        
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
        
        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]      
            
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"].str.upper() == "PRESIDENT"]
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        general_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(general_df_list, ignore_index=True)

# Checking any suspicious precinct
suspicious_precincts = gen_combined_df[
    gen_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]
general_files


['C:\\Huy Phan\\College\\VoterTurnout\\data\\SC\\20161108__sc__general__precinct.csv']

In [16]:
# Select only the relevant columns
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]

general_data

,precinct,party,candidate,votes
0,ABBEVILLEABBEVILLE NO. 1,DEM,Hillary Rodham Clinton / Timothy Michael Kaine,245
1,ABBEVILLEABBEVILLE NO. 2,DEM,Hillary Rodham Clinton / Timothy Michael Kaine,373
2,ABBEVILLEABBEVILLE NO. 3,DEM,Hillary Rodham Clinton / Timothy Michael Kaine,214
3,ABBEVILLEABBEVILLE NO. 4,DEM,Hillary Rodham Clinton / Timothy Michael Kaine,131
4,ABBEVILLEANTREVILLE,DEM,Hillary Rodham Clinton / Timothy Michael Kaine,105
...,...,...,...,...
14682,YORKFAILSAFE 2,LIB,Gary Johnson / Bill Weld,1
14683,YORKPROVISIONAL 1,LIB,Gary Johnson / Bill Weld,3
14684,YORKPROVISIONAL 2,LIB,Gary Johnson / Bill Weld,4
14685,YORKPROVISIONAL 3,LIB,Gary Johnson / Bill Weld,1


In [17]:
# Viewing candidate data
general_data["candidate"].value_counts(dropna=False)

candidate
Donald J Trump / Michael R Pence                  2456
Hillary Rodham Clinton / Timothy Michael Kaine    2455
Gary Johnson / Bill Weld                          2310
Evan McMullin / Nathan Johnson                    2161
Jill Stein / Ajamu Baraka                         2108
Darrell Castle / Scott Bradley                    1736
Peter Skewes / Michael Lacy                       1461
Name: count, dtype: int64

In [18]:
# Cleaning Candidates

# Turning all general data to uppercase
general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"\w*WRITE[\s-]\w*",
    r"NAN",
    r"UNCERTIFIED",
    r"UNVERIFIED",
    r"NONE OF THE ABOVE",
    r"LBT",
    r"DEM",
    r"REP",
    r"GRN",
    r"NAN"

]

# Fixing FUENTE
general_data.loc[
    general_data["candidate"].str.contains("\w*Fuenta\w*", case=False, na=False),
    "candidate"
] = "FUENTE"
pattern = "|".join(unwanted_keywords)

# # FILTER OUT TRASH WORDS
general_data = general_data[~general_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

# # Fixing Darrell Castle"
# general_data.loc[
#     general_data["candidate"].str.contains("\w*ASTLE\w*", case=False, na=False),
#     "candidate"
# ] = "CASTLE"

# general_data.loc[
#     general_data["candidate"].str.contains("\w*ATUREN\w*", case=False, na=False),
#     "candidate"
# ] = "MATUREN"

# Fixing mixed president + vice_president by /
general_data["candidate"] = (
    general_data["candidate"].str.split("/")
    .str[0]
    .str.strip()
    )

# #Selecting only last name
# general_data["candidate"] = general_data["candidate"].str.split().str[-1]

# # Fixing the , candidate
# # general_data["candidate"] = (
# #     general_data["candidate"].str.split(",")
# #     .str[0]
# #     .str.strip()
# #     )




# # Fixing Cubbler
# general_data.loc[
#     ( 
#         general_data["candidate"].str.contains("\w*UBBIER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*UBLER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*OBBLER\w*", case=False, na=False),
#     "candidate")
# ] = "CUBBLER"


# # Fixing Kotlikoff
# general_data.loc[
#     general_data["candidate"].str.contains("\w*TIKOFF\w*", case=False, na=False),
#     "candidate"
# ] = "KOTLIKOFF"

# # Fixing Valdivia
# general_data.loc[
#     general_data["candidate"].str.contains("\w*VALDIVA\W*", case=False, na=False),
#     "candidate"
# ] = "VALDIVIA"

# # Fixing HOEFLING
# general_data.loc[
#     general_data["candidate"].str.contains("\w*HOEFFLING", case=False, na=False),
#     "candidate"
# ] = "HOEFLING"


# # Fixing McMullin
# general_data.loc[
#     general_data["candidate"].str.contains("MCMULLEN", case=False, na=False) ,
#     "candidate"
# ] = "MCMULLIN"

# Fixing LASTNAME + First name Initial
# general_data.loc[:,"candidate"] = (
#     general_data["candidate"].str.split().str[0].str.upper()
# )


# #Selecting only last name
general_data["candidate"] = general_data["candidate"].str.split().str[-1]

general_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34304\3810900807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


candidate
TRUMP       2456
CLINTON     2455
JOHNSON     2310
MCMULLIN    2161
STEIN       2108
CASTLE      1736
SKEWES      1461
Name: count, dtype: int64

In [19]:
# Viewing Party
general_data["party"].value_counts(dropna=False)

party
REP    2456
DEM    2455
LIB    2310
IND    2161
GRN    2108
CON    1736
AMR    1178
AME     150
AMC     133
Name: count, dtype: int64

In [20]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown
    

#=====================================
# Function to fill remaining missing party from master lookup
#=====================================
def fill_party_from_master(df, master_df):
    party_map = master_df.set_index("candidate")["party"].to_dict()
    df["party"] = df.apply(
        lambda row: party_map.get(row["candidate"], row["party"])
        if pd.isna(row["party"]) else row["party"],
        axis=1
    )
    return df


#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)

    # Filter and combine only known party entries (exclude 'UNK')
    new_data = (
        df[["candidate", "party"]]
        .dropna()
        .query('party != "UNK"')
        .drop_duplicates()
    )

    # Merge with master and remove duplicates by candidate
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
# general_data["party"] = general_data.apply(
#     lambda row: fill_party_from_data(row, general_data),
#     axis=1
# )

# # # Fill remaining party using general master CSV
# master_party_df = pd.read_csv(r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv") # USE YOUR OWN ADDRESS
# general_data = fill_party_from_master(general_data, master_party_df)
general_data["party"].value_counts(dropna=False)


party
REP    2456
DEM    2455
LIB    2310
IND    2161
GRN    2108
CON    1736
AMR    1178
AME     150
AMC     133
Name: count, dtype: int64

In [21]:
# Cleaning Party
# Turning all general data party to uppercase
general_data["party"] = general_data["party"].astype(str).str.upper()

general_data["party"] = (
    general_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEMOCRAT":"DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GRN",
        "GREEN AND RAINBOW":"GRN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LIBERTARIN":"LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "AMERICAN DELTA":"AMD",
        "PROHIBITION":"PRO",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK",
        "NPA":"IND",
        "UST":"CON",
        "(WRITE-IN)":"IND"
    })
)

general_data["party"].value_counts(dropna=False)

party
REP    2456
DEM    2455
LIB    2310
IND    2161
GRN    2108
CON    1736
AMR    1178
AME     150
AMC     133
Name: count, dtype: int64

In [22]:
# UPDATE MASTER FILE, CAREFUL
update_master_candidate_party(general_data, r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv")

In [23]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

candidate_column,precinct,gen_amc_SKEWES,gen_ame_SKEWES,gen_amr_SKEWES,gen_con_CASTLE,gen_dem_CLINTON,gen_grn_STEIN,gen_ind_MCMULLIN,gen_lib_JOHNSON,gen_rep_TRUMP
0,ABBEVILLEABBEVILLE NO. 1,0,0,2,2,245,1,4,21,610
1,ABBEVILLEABBEVILLE NO. 2,0,0,1,1,373,0,2,7,303
2,ABBEVILLEABBEVILLE NO. 3,0,0,2,3,214,2,3,5,284
3,ABBEVILLEABBEVILLE NO. 4,0,0,0,2,131,1,2,5,279
4,ABBEVILLEABSENTEE,0,0,2,12,1270,7,8,29,1439
...,...,...,...,...,...,...,...,...,...,...
2459,YORKWATERSTONE,2,0,0,4,419,11,16,44,602
2460,YORKWINDJAMMER,2,0,0,5,382,8,12,41,699
2461,YORKWYLIE,1,0,0,4,168,2,5,25,411
2462,YORKYORK NO. 1,1,0,0,0,405,7,7,14,309


In [24]:
# Merge
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")

# Convert DEM primary columns to numeric and calculate total
dem_cols = combined.filter(like="pri_dem_").columns
combined[dem_cols] = combined[dem_cols].apply(pd.to_numeric, errors="coerce")
combined["dem_primary_total"] = combined[dem_cols].sum(axis=1)

# Convert general election columns to numeric and calculate total
gen_cols = combined.filter(like="gen_").columns
combined[gen_cols] = combined[gen_cols].apply(pd.to_numeric, errors="coerce")
combined["general_total"] = combined[gen_cols].sum(axis=1)

# Convert all numeric columns to int
for col in combined.columns[1:]:
    combined[col] = pd.to_numeric(combined[col], errors='coerce').fillna(0).astype(int)

# Identify filtered-out precincts
primary_precincts = set(primary_result["precinct"])
general_precincts = set(general_result["precinct"])
combined_precincts = set(combined["precinct"])

primary_filtered_out = primary_precincts - combined_precincts
general_filtered_out = general_precincts - combined_precincts




In [25]:


combined.to_csv("SC.csv", index=False)
# Step 3: Identify unmatched precincts and retrieve full rows
combined_precincts = set(combined["precinct"])

# Rows in primary_result but not in combined
primary_filtered_out = primary_result[~primary_result["precinct"].isin(combined_precincts)]
primary_filtered_out.to_csv("SC_primary_filtered.csv", index=False)

# Rows in general_result but not in combined
general_filtered_out = general_result[~general_result["precinct"].isin(combined_precincts)]
general_filtered_out.to_csv("SC_general_filtered.csv", index=False)

In [26]:
print(f"primary: {len(primary_precincts)}, general: {len(general_precincts)}, combined: {len(combined)}")

primary: 2455, general: 2464, combined: 2296


In [27]:
print(f"primary: {len(pri_combined_df['county'].unique())}, general: {len(gen_combined_df['county'].unique())}")


primary: 46, general: 46


In [28]:
pri_combined_df['precinct'].unique()

array(['SALUDACENTENNIAL', 'SALUDACLYDE', 'SALUDADELMAR', ...,
       'BEAUFORTFAILSAFE', 'BEAUFORTPROVISIONAL',
       'BEAUFORTFAILSAFE PROVISIONAL'], dtype=object)

In [29]:
gen_combined_df['precinct'].unique()

array(['ABBEVILLEABBEVILLE NO. 1', 'ABBEVILLEABBEVILLE NO. 2',
       'ABBEVILLEABBEVILLE NO. 3', ..., 'YORKPROVISIONAL 3',
       'YORKPROVISIONAL 4', 'YORKFAILSAFE PROV 1'], dtype=object)

In [30]:
primary_counties = set(pri_combined_df['precinct'].dropna().str.strip().str.upper())
general_counties = set(gen_combined_df['precinct'].dropna().str.strip().str.upper())

diff = primary_counties - general_counties
print(f"Counties in primary but not in general: {len(diff)}")
print(diff)


Counties in primary but not in general: 159
{'BEAUFORTBLUFF 5A', 'LEXINGTONGASTON 1', 'DILLONPROVISIONAL', 'YORKEMERGENCY', 'LEXINGTONPINE RIDGE 1', 'CALHOUNEMERGENCY', 'YORKFAILSAFE', 'SUMTEREMERGENCY', 'LEXINGTONWEST COLUMBIA NO. 3', 'LEXINGTONLEAPHART RD', 'LEXINGTONMT. HEBRON', 'BEAUFORTLI 3A', 'COLLETONPROVISIONAL', 'LEXINGTONGASTON 2', 'LEXINGTONEDMUND 2', 'HORRYNORTON', 'BEAUFORTBLUFF 4C', 'BEAUFORTLI 2C', 'MCCORMICKEMERGENCY', 'JASPEREMERGENCY', 'FLORENCEEMERGENCY 1', 'BEAUFORTLI 2A', 'LEXINGTONPILGRIM CHRUCH', 'COLLETONEMERGENCY', 'SUMTERABSENTEE', 'LEXINGTONCONGAREE 2', 'ALLENDALEFAILSAFE', 'BEAUFORTBLUFF 2D', 'JASPERFAILSAFE/PROVISIONAL', 'BEAUFORTLI 3C', 'CHARLESTONEMERGENCY', 'BEAUFORTBLUFF 1A', 'COLLETONFAILSAFE', 'UNIONABSENTEE', 'SALUDAFAILSAFE PROVISIONAL', 'BEAUFORTST. HELENA 2A', 'GREENWOODEMERGENCY', 'YORKPROVISIONAL', 'CHESTERFIELDFAILSAFE', 'DILLONFAILSAFE PROVISIONAL', "LANCASTERGOOCH'S CROSS RDS", 'SUMTERPROVISIONAL', 'MARLBOROEMERGENCY', 'KERSHAWABSENTEE', 'LAN